# Models

In [258]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import datetime

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import Imputer

import statsmodels.api as sm
from statsmodels.api import OLS

In [214]:
def display_df(df, nrows=5, ncols=None):
    with pd.option_context('display.max_rows', nrows, 'display.max_columns', ncols):
        display (df)
#gets ratio of NaNs for each column
def stats_NaN(df):
    df_stats = pd.DataFrame(index=[df.columns], columns=["NaN Ratio"])
    for col in df.columns:
        df_stats["NaN Ratio"][col] = df[col].isna().sum()/len(df) #NaN ratio
    return df_stats.sort_values(by=['NaN Ratio'])

In [102]:
df_whole = pd.read_csv("../data/data_clean/clean_accepted_2007_to_2018Q2.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (38,48,107,118,119,120,123,124,125,128,134,135,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [104]:
stats_nan = stats_NaN(df_whole)

In [105]:
display_df(stats_nan, None)

,NaN Ratio
addr_state_DC,0
purpose_debt_consolidation,0
purpose_educational,0
purpose_home_improvement,0
purpose_house,0
purpose_major_purchase,0
purpose_medical,0
purpose_moving,0
purpose_other,0
purpose_credit_card,0


In [106]:
our_drop_list = ['funded_amnt','funded_amnt_inv','int_rate','installment','grade','sub_grade',
                 'pymnt_plan','zip_code','initial_list_status','out_prncp', 'application_type', 'policy_code',
                 'out_prncp_inv','total_pymnt','total_pymnt_inv','total_rec_prncp','total_rec_int',
                 'total_rec_late_fee','recoveries','collection_recovery_fee','last_pymnt_d','last_pymnt_amnt',
                 'next_pymnt_d','last_credit_pull_d','last_fico_range_high','last_fico_range_low',
                 'collections_12_mths_ex_med','mths_since_last_major_derog','acc_now_delinq','tot_coll_amt',
                 'tot_cur_bal','open_acc_6m','open_act_il','open_il_12m','open_il_24m','mths_since_rcnt_il',
                 'total_bal_il','il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','total_rev_hi_lim',
                 'inq_fi','total_cu_tl','inq_last_12m','acc_open_past_24mths','avg_cur_bal','bc_open_to_buy',
                 'bc_util','chargeoff_within_12_mths','delinq_amnt','mo_sin_old_il_acct','mo_sin_old_rev_tl_op',
                 'mo_sin_rcnt_rev_tl_op','mo_sin_rcnt_tl','mort_acc','mths_since_recent_bc',
                 'mths_since_recent_bc_dlq','mths_since_recent_inq','mths_since_recent_revol_delinq',
                 'num_accts_ever_120_pd','num_actv_bc_tl','num_actv_rev_tl','num_bc_sats','num_bc_tl',
                 'num_il_tl','num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m',
                 'num_tl_30dpd','num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','percent_bc_gt_75',
                 'pub_rec_bankruptcies','tax_liens','tot_hi_cred_lim','total_bal_ex_mort','total_bc_limit',
                 'total_il_high_credit_limit','revol_bal_joint','sec_app_fico_range_high',
                 'sec_app_earliest_cr_line','sec_app_inq_last_6mths','sec_app_mort_acc','sec_app_open_acc',
                 'sec_app_revol_util','sec_app_open_act_il','sec_app_num_rev_accts',
                 'sec_app_chargeoff_within_12_mths','sec_app_collections_12_mths_ex_med',
                 'sec_app_mths_since_last_major_derog','hardship_flag','hardship_type','hardship_reason',
                 'hardship_status','deferral_term','hardship_amount','hardship_start_date','hardship_end_date',
                 'payment_plan_start_date','hardship_length','hardship_dpd','hardship_loan_status',
                 'orig_projected_additional_accrued_interest','hardship_payoff_balance_amount',
                 'hardship_last_payment_amount','disbursement_method','debt_settlement_flag',
                 'debt_settlement_flag_date','settlement_status','settlement_date','settlement_amount',
                 'settlement_percentage','settlement_term']

In [134]:
df_less_feats = df_whole.drop(columns=our_drop_list)

In [135]:
display_df(df_less_feats)

,loan_amnt,term,emp_length,annual_inc,issue_d,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,annual_inc_joint,dti_joint,verification_status_joint,sec_app_fico_range_low,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Does not meet the credit policy. Status:Charged Off,loan_status_Does not meet the credit policy. Status:Fully Paid,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,num_grade
0,15000.0,1.0,10.0,78000.0,2014-12-01,0.1203,0.0,1994-08-01,750.0,754.0,0.0,800.0,800.0,6.0,0.0,138008.0,29.0,17.0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2.0
1,10400.0,0.0,8.0,58000.0,2014-12-01,0.1492,0.0,1989-09-01,710.0,714.0,2.0,42.0,800.0,17.0,0.0,6133.0,31.6,36.0,NaN,NaN,NaN,NaN,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4
2,21425.0,1.0,6.0,63800.0,2014-12-01,0.1849,0.0,2003-08-01,685.0,689.0,0.0,60.0,800.0,10.0,0.0,16374.0,76.2,35.0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0
3,12800.0,1.0,10.0,125000.0,2014-12-01,0.0831,1.0,2000-10-01,665.0,669.0,0.0,17.0,800.0,8.0,0.0,5753.0,100.9,13.0,NaN,NaN,NaN,NaN,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.6
4,7650.0,0.0,0.5,50000.0,2014-12-01,0.3481,0.0,2002-08-01,685.0,689.0,1.0,800.0,800.0,11.0,0.0,16822.0,91.9,20.0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004084,18000.0,0.0,10.0,47000.0,2018-01-01,0.1537,0.0,1992-08-01,680.0,684.0,0.0,33.0,800.0,13.0,0.0,11362.0,39.0,29.0,NaN,NaN,NaN,NaN,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## Data Augmentation

There might be some value in having a secondary applicant. We will create a new variable from a check on `sec_app_fico_range_low` to detect a secondary applicant. We will also drop the other high NaN proportion features.

In [138]:
def add_secondary(df, label='sec_app_fico_range_low'):
    df['secondary'] = df[label].apply(lambda x: int(not pd.isnull(x)))

In [139]:
add_secondary(df_less_feats)

In [140]:
drop_high_nan_feats = ['annual_inc_joint', 'dti_joint','verification_status_joint','sec_app_fico_range_low']
df_less_feats.drop(columns=drop_high_nan_feats, inplace=True)

In [141]:
display_df(df_less_feats)

,loan_amnt,term,emp_length,annual_inc,issue_d,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,home_ownership_ANY,home_ownership_MORTGAGE,home_ownership_NONE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,verification_status_Not Verified,verification_status_Source Verified,verification_status_Verified,loan_status_Charged Off,loan_status_Current,loan_status_Default,loan_status_Does not meet the credit policy. Status:Charged Off,loan_status_Does not meet the credit policy. Status:Fully Paid,loan_status_Fully Paid,loan_status_In Grace Period,loan_status_Late (16-30 days),loan_status_Late (31-120 days),purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_educational,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding,addr_state_AK,addr_state_AL,addr_state_AR,addr_state_AZ,addr_state_CA,addr_state_CO,addr_state_CT,addr_state_DC,addr_state_DE,addr_state_FL,addr_state_GA,addr_state_HI,addr_state_IA,addr_state_ID,addr_state_IL,addr_state_IN,addr_state_KS,addr_state_KY,addr_state_LA,addr_state_MA,addr_state_MD,addr_state_ME,addr_state_MI,addr_state_MN,addr_state_MO,addr_state_MS,addr_state_MT,addr_state_NC,addr_state_ND,addr_state_NE,addr_state_NH,addr_state_NJ,addr_state_NM,addr_state_NV,addr_state_NY,addr_state_OH,addr_state_OK,addr_state_OR,addr_state_PA,addr_state_RI,addr_state_SC,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY,num_grade,secondary
0,15000.0,1.0,10.0,78000.0,2014-12-01,0.1203,0.0,1994-08-01,750.0,754.0,0.0,800.0,800.0,6.0,0.0,138008.0,29.0,17.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2.0,0
1,10400.0,0.0,8.0,58000.0,2014-12-01,0.1492,0.0,1989-09-01,710.0,714.0,2.0,42.0,800.0,17.0,0.0,6133.0,31.6,36.0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.4,0
2,21425.0,1.0,6.0,63800.0,2014-12-01,0.1849,0.0,2003-08-01,685.0,689.0,0.0,60.0,800.0,10.0,0.0,16374.0,76.2,35.0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.0,0
3,12800.0,1.0,10.0,125000.0,2014-12-01,0.0831,1.0,2000-10-01,665.0,669.0,0.0,17.0,800.0,8.0,0.0,5753.0,100.9,13.0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.6,0
4,7650.0,0.0,0.5,50000.0,2014-12-01,0.3481,0.0,2002-08-01,685.0,689.0,1.0,800.0,800.0,11.0,0.0,16822.0,91.9,20.0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2004084,18000.0,0.0,10.0,47000.0,2018-01-01,0.1537,0.0,1992-08-01,680.0,684.0,0.0,33.0,800.0,13.0,0.0,11362.0,39.0,29.0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.4,0
2004085,6000.0,0.0,10.0,50000.0,2018-01-01,0.2893,0.0,1997-06-01,690.0,694.0,0.0,58.0,800.0,11.0,0.0,6950.0,51.9,14.0,0,1

As oppose to a range for FICO scores, we think it would be better to model on the average.

In [142]:
df_less_feats['fico_avg'] = df_less_feats[['fico_range_low', 'fico_range_high']].mean(axis=1)
df_less_feats.drop(columns=['fico_range_low', 'fico_range_high'], inplace=True)

`earliest_cr_line` may not be itself a useful feature (a date with no context). However it can tell us how long a person has had a credit line open, which is probably useful.

In [143]:
def timedelta_to_day(t):
    if not pd.isnull(t):
        if isinstance(t, int):
            return t
        else:
            return t.days
    else:
        return np.nan

In [144]:
df_less_feats['issue_d'] = pd.to_datetime(df_less_feats['issue_d'])
df_less_feats['earliest_cr_line'] = pd.to_datetime(df_less_feats['earliest_cr_line'])
df_less_feats['cr_line_hist'] = df_less_feats['issue_d'] - pd.to_datetime(df_less_feats['earliest_cr_line'])
df_less_feats['cr_line_hist'] = df_less_feats['cr_line_hist'].apply(timedelta_to_day)

In [161]:
df_less_feats.drop(columns=['earliest_cr_line'],inplace=True)

## Data Prep

We need to deal with NaNs. We can drop the samples for NaN features when the NaN ratio of that feature is small. For the rest, which is just `emp_length`, we'll do mean imputation to keep things simple.

In [154]:
stats_nan_less = stats_NaN(df_less_feats)

In [155]:
display_df(stats_nan_less,None)

,NaN Ratio
addr_state_AZ,0
addr_state_IL,0
addr_state_ID,0
addr_state_IA,0
addr_state_HI,0
addr_state_GA,0
addr_state_FL,0
addr_state_DE,0
addr_state_DC,0
addr_state_IN,0


In [170]:
df_less_feats.dropna(subset=['issue_d','annual_inc','delinq_2yrs','total_acc','open_acc','pub_rec','cr_line_hist',
               'inq_last_6mths','dti','revol_util'],inplace=True)

In [171]:
stats_nan_less = stats_NaN(df_less_feats)
display_df(stats_nan_less,None)

,NaN Ratio
loan_amnt,0
addr_state_MT,0
addr_state_MS,0
addr_state_MO,0
addr_state_MN,0
addr_state_MI,0
addr_state_ME,0
addr_state_MD,0
addr_state_MA,0
addr_state_LA,0


Most models can't handle datetime objects. So we will convert this to a float.

In [238]:
df_less_feats['issue_d']=df_less_feats['issue_d'].map(datetime.datetime.toordinal)

TypeError: descriptor 'toordinal' requires a 'datetime.date' object but received a 'int'

In [239]:
df_train, df_test = train_test_split(df_less_feats, random_state=9001, test_size=0.2)

In [240]:
cols_to_imp = ['emp_length']

In [241]:
imp_mean = Imputer(copy=True, missing_values=np.nan, strategy='mean').fit(df_train[cols_to_imp]) #fit to training data
df_train[cols_to_imp] = imp_mean.transform(df_train[cols_to_imp])
df_test[cols_to_imp] = imp_mean.transform(df_test[cols_to_imp])

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexin

In [242]:
display_df(stats_NaN(df_train),None) #should be all zeros

,NaN Ratio
loan_amnt,0
addr_state_MT,0
addr_state_MS,0
addr_state_MO,0
addr_state_MN,0
addr_state_MI,0
addr_state_ME,0
addr_state_MD,0
addr_state_MA,0
addr_state_LA,0


In [243]:
target = 'num_grade'
x_cols = list(set([target]).symmetric_difference(list(df_train.columns)))
x_train, x_test = df_train[x_cols], df_test[x_cols]
y_train, y_test = df_train[target], df_test[target]

Now we can standardize the data.

In [244]:
def standardize(x, x_ref, labels):
    std = np.std(x_ref[labels])
    mean = np.mean(x_ref[labels])
    x_std = (x[labels] - mean)/std
    return x_std

In [245]:
std_labels = ['loan_amnt','total_acc','revol_util','revol_bal','pub_rec','open_acc','mths_since_last_record',
             'mths_since_last_delinq','inq_last_6mths','delinq_2yrs','dti','annual_inc','fico_avg',
             'cr_line_hist','emp_length','issue_d']
x_train_unstand = x_train.copy()
x_train_std = x_train.copy()
x_train_std[std_labels] = standardize(x_train, x_train_unstand, std_labels)

x_test_unstand = x_test.copy()
x_test_std = x_test.copy()
x_test_std[std_labels] = standardize(x_test, x_train_unstand, std_labels)

In [247]:
display_df(x_test_std, 10)

,home_ownership_NONE,home_ownership_OWN,addr_state_VT,addr_state_NJ,issue_d,verification_status_Not Verified,loan_status_Late (31-120 days),purpose_home_improvement,addr_state_CT,addr_state_AR,loan_status_Current,addr_state_OK,loan_status_Charged Off,loan_status_Does not meet the credit policy. Status:Charged Off,addr_state_NE,addr_state_RI,addr_state_MI,addr_state_CA,addr_state_MS,pub_rec,addr_state_KY,annual_inc,delinq_2yrs,loan_status_Fully Paid,addr_state_IA,open_acc,addr_state_WI,emp_length,purpose_educational,term,loan_amnt,dti,addr_state_DE,addr_state_HI,addr_state_NM,addr_state_KS,addr_state_IL,mths_since_last_record,addr_state_PA,revol_bal,purpose_credit_card,addr_state_WA,addr_state_OH,revol_util,addr_state_TN,secondary,home_ownership_OTHER,addr_state_SC,loan_status_Does not meet the credit policy. Status:Fully Paid,inq_last_6mths,addr_state_ID,loan_status_Late (16-30 days),verification_status_Verified,addr_state_WV,purpose_moving,cr_line_hist,fico_avg,addr_state_VA,addr_state_GA,addr_state_AL,addr_state_MN,home_ownership_ANY,addr_state_AK,purpose_debt_consolidation,purpose_medical,addr_state_FL,addr_state_LA,home_ownership_MORTGAGE,addr_state_NC,loan_status_In Grace Period,addr_state_ME,purpose_car,addr_state_IN,addr_state_MA,loan_status_Default,addr_state_MT,addr_state_DC,addr_state_OR,addr_state_ND,purpose_major_purchase,addr_state_AZ,total_acc,addr_state_MD,purpose_vacation,addr_state_UT,addr_state_WY,mths_since_last_delinq,addr_state_NV,purpose_house,addr_state_TX,purpose_renewable_energy,addr_state_SD,verification_status_Source Verified,addr_state_NH,purpose_wedding,addr_state_MO,addr_state_CO,purpose_small_business,home_ownership_RENT,addr_state_NY,purpose_other
1313171,0,0,0,0,0.276766,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.348420,0,-0.059762,-0.36003,1,0,-0.647877,0,1.132285e+00,0,0.0,-1.184117,0.501973,0,0,0,0,0,0.442425,0,0.203438,0,0,0,-0.260427,0,0,0,0,0,-0.658467,0,0,0,0,0,-0.395739,0.531778,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.532082,0,0,0,0,0.986612,0,0,0,0,0,0,0,0,0,0,0,0,0,0
996310,0,0,0,0,0.671426,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1.331765,0,-0.224816,-0.36003,0,0,0.066499,0,-5.476576e-13,0,0.0,-0.322560,-0.285239,0,0,0,0,0,-2.327242,0,-0.247229,0,0,0,-0.931060,0,0,0,0,0,-0.658467,0,0,0,0,0,1.748912,-0.235208,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.722436,0,0,0,0,-0.921277,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6787,0,0,0,0,-0.661974,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.331765,0,-0.224816,-0.36003,1,0,1.138062,0,-8.531117e-01,0,0.0,-0.783714,0.313133,0,0,0,0,0,-2.227267,0,-0.543966,0,0,0,0.503688,0,0,0,0,0,-0.658467,0,0,0,0,0,-0.362486,-1.002194,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1.140609,0,0,0,0,-0.923887,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1431802,0,0,0,0,0.869569,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.348420,0,-0.059296,-0.36003,0,0,1.138062,0,-5.476576e-13,0,0.0,1.665647,0.963076,0,0,0,0,0,0.442425,0,1.746530,0,0,0,0.686588,0,0,0,0,0,0.446710,0,0,1,0,0,1.454640,0.224984,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.140609,0,0,0,0,0.986612,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1675772,0,0,0,0,1.118059,1,0,1,0,0,1,0,0,0,0,0,0,0,0,-0.348420,0,0.855062,-0.36003,0,0,0.066499,0,-2.227245e-03,0,0.0,-0.543472,-1.082311,0,0,0,0,0,0.442425,0,-0.345196,0,0,0,-0.699387,0,0,0,0,0,0.446710,0,0,0,0,0,-0.536618,-0.081810,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,-0.448447,0,0,0,0,-0.923887,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1269664,0,0,0,0,0.327114,0,0,0,0,0,1,0,0,0,0,0,0,0,0,-0.348420,0,0.503882,-0.36003,0,0,-0.112095,0,5.650291e-01,0,0.0,1.886559,-0.566603,0,0,0,0,0,0.442425,0,-0.244350,0,0,0,-1.280602,0,0,0,1,0,-0.658467,0,0

## Models

### Baseline Model - Linear Regression (OLS)

In [248]:
X_train = sm.add_constant(x_train)
X_test = sm.add_constant(x_test)
model_OLS = OLS(y_train, X_train.astype(float)).fit()

In [255]:
OLS_train_pred = model_OLS.predict(X_train)
OLS_test_pred = model_OLS.predict(X_test)
score_OLS_train = r2_score(y_train,OLS_train_pred)
score_OLS_test = r2_score(y_test, OLS_test_pred)

In [256]:
print('Score on the training set is %.6f'%score_OLS_train)
print('Score on the test set is %.6f'%score_OLS_test)

Score on the training set is 0.484437
Score on the test set is 0.484159


In [260]:
mpl.rcParams['agg.path.chunksize'] = 10000
plt.scatter(OLS_test_pred)

TypeError: scatter() missing 1 required positional argument: 'y'